In [1]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# # Load ArUco dictionary
# dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
# parameters = cv2.aruco.DetectorParameters()

# # Open the main video file
# cap_main = cv2.VideoCapture("video.mp4")

# while cap_main.isOpened():
#     ret, frame = cap_main.read()
#     if not ret:
#         break  # Stop when video ends

#     # Convert to grayscale for marker detection
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     corners, ids, _ = cv2.aruco.detectMarkers(gray, dictionary, parameters=parameters)

#     if ids is not None and len(ids) >= 4:
#         print("Detected Marker IDs:", ids.flatten())

#         # Extract ROI corner points
#         roi_corners = {}
#         for marker_id, corner_set in zip(ids.flatten(), corners):
#             roi_corners[marker_id] = corner_set[0]

#         # Ensure all 4 required markers exist
#         required_ids = [23, 25, 30, 33]
#         if all(marker_id in roi_corners for marker_id in required_ids):
#             ref_pt1 = roi_corners[23][0]  # Top-left
#             ref_pt2 = roi_corners[25][1]  # Top-right
#             ref_pt3 = roi_corners[30][2]  # Bottom-right
#             ref_pt4 = roi_corners[33][3]  # Bottom-left

#             # Define ROI points
#             pts_dst = np.array([
#                 [ref_pt1[0], ref_pt1[1]],
#                 [ref_pt2[0], ref_pt2[1]],
#                 [ref_pt3[0], ref_pt3[1]],
#                 [ref_pt4[0], ref_pt4[1]]
#             ], dtype=np.float32)

#             print("Destination ROI Points:", pts_dst)

#     # Display detected markers
#     frame_detected = frame.copy()
#     cv2.aruco.drawDetectedMarkers(frame_detected, corners, ids)
    
#     plt.figure(figsize=[10, 10])
#     plt.axis('off')
#     plt.imshow(cv2.cvtColor(frame_detected, cv2.COLOR_BGR2RGB))
#     plt.show()

# cap_main.release()
# cv2.destroyAllWindows()


In [3]:
# Open the source video (piano.mp4)
cap_piano = cv2.VideoCapture("piano.mp4")

if not cap_piano.isOpened():
    print("Error: Could not open piano.mp4.")
    exit()

# Read the first frame
ret_piano, frame_piano = cap_piano.read()

if not ret_piano or frame_piano is None:
    print("Error: Could not read a valid frame from piano.mp4.")
    cap_piano.release()
    exit()

# Compute detected ROI dimensions
roi_width = int(np.linalg.norm(ref_pt1 - ref_pt2))
roi_height = int(np.linalg.norm(ref_pt1 - ref_pt3))

# Compute aspect ratios
roi_aspect_ratio = roi_width / roi_height
src_height, src_width, _ = frame_piano.shape
src_aspect_ratio = src_width / src_height

# Crop source video to match aspect ratio
if src_aspect_ratio > roi_aspect_ratio:
    new_width = int(src_height * roi_aspect_ratio)
    crop_x = (src_width - new_width) // 2
    frame_piano = frame_piano[:, crop_x:crop_x + new_width]
elif src_aspect_ratio < roi_aspect_ratio:
    new_height = int(src_width / roi_aspect_ratio)
    crop_y = (src_height - new_height) // 2
    frame_piano = frame_piano[crop_y:crop_y + new_height, :]

# Resize after cropping
frame_piano = cv2.resize(frame_piano, (roi_width, roi_height))

# Define Source Points
pts_src = np.array([
    [0, 0], 
    [roi_width, 0], 
    [roi_width, roi_height], 
    [0, roi_height]
], dtype=np.float32)

print("Source Video Points:", pts_src)

cap_piano.release()


Source Video Points: [[  0.   0.]
 [637.   0.]
 [637. 844.]
 [  0. 844.]]


In [4]:
# Open the main video file
cap_main = cv2.VideoCapture("video.mp4")

# Check if video is opened successfully
if not cap_main.isOpened():
    print("Error: Could not open video.mp4. Check if the file exists and is playable.")
    exit()

# Read the first frame
ret_main, frame = cap_main.read()

if not ret_main or frame is None:
    print("Error: Could not read a valid frame from video.mp4.")
    cap_main.release()
    exit()

cap_main.release()


In [5]:
if frame_piano is None:
    print("Error: Source video frame is not loaded correctly.")
    exit()

In [6]:
# Ensure both frames exist
if frame is not None and frame_piano is not None:
    warped_piano = cv2.warpPerspective(frame_piano, M, (frame.shape[1], frame.shape[0]))

    # Display warped video
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(warped_piano, cv2.COLOR_BGR2RGB))
    plt.title("Warped Source Video")
    plt.axis("off")
    plt.show()
else:
    print("Error: Either frame or frame_piano is None. Cannot proceed with warping.")


NameError: name 'M' is not defined

In [ ]:
print("frame:", frame)
print("frame_piano:", frame_piano)


frame: [[[198 198 198]
  [198 198 198]
  [198 198 198]
  ...
  [122 141 140]
  [122 141 140]
  [123 143 141]]

 [[198 198 198]
  [198 198 198]
  [198 198 198]
  ...
  [122 141 140]
  [122 141 140]
  [122 141 140]]

 [[198 198 198]
  [198 198 198]
  [198 198 198]
  ...
  [120 140 139]
  [122 141 140]
  [122 141 140]]

 ...

 [[192 188 186]
  [192 188 186]
  [190 187 185]
  ...
  [  5  23  39]
  [  5  23  39]
  [  5  23  39]]

 [[193 189 187]
  [192 188 186]
  [190 187 185]
  ...
  [  2  19  35]
  [  1  18  34]
  [  1  18  34]]

 [[193 189 187]
  [192 188 186]
  [190 187 185]
  ...
  [  0  16  32]
  [  0  15  31]
  [  0  15  31]]]
frame_piano: [[[ 79  82  79]
  [ 79  82  79]
  [ 79  82  79]
  ...
  [103 108 100]
  [103 108 100]
  [103 108 100]]

 [[ 79  82  79]
  [ 79  82  79]
  [ 79  82  79]
  ...
  [103 108 100]
  [103 108 100]
  [103 108 100]]

 [[ 79  82  79]
  [ 79  82  79]
  [ 79  82  79]
  ...
  [103 108 100]
  [103 108 100]
  [103 108 100]]

 ...

 [[  4   1   5]
  [  4   1   5]


In [ ]:
# Compute the transformation matrix
M = cv2.getPerspectiveTransform(pts_src, pts_dst)

# Warp the source video frame
warped_piano = cv2.warpPerspective(frame_piano, M, (frame.shape[1], frame.shape[0]))

# Display the warped video frame
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(warped_piano, cv2.COLOR_BGR2RGB))
plt.title("Warped Source Video")
plt.axis("off")
plt.show()


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# Create a mask for blending
mask = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.uint8)
cv2.fillConvexPoly(mask, np.int32([pts_dst]), (255, 255, 255), cv2.LINE_AA)

# Convert mask to 3 channels
mask3 = np.zeros_like(frame, dtype=np.uint8)
for i in range(3):
    mask3[:, :, i] = mask

# Blend images
frame_out = cv2.bitwise_and(frame, cv2.bitwise_not(mask3))  # Remove original region
frame_out += cv2.bitwise_and(warped_piano, mask3)  # Overlay warped video

# Display the final result
cv2.imshow("Warped Video Overlay", frame_out)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# Open the main video and source video
cap_main = cv2.VideoCapture("video.mp4")
cap_piano = cv2.VideoCapture("piano.mp4")

# Check if both videos opened successfully
if not cap_main.isOpened() or not cap_piano.isOpened():
    print("Error: Could not open one or both video files.")
    exit()

while cap_main.isOpened() and cap_piano.isOpened():
    ret_main, frame_main = cap_main.read()
    ret_piano, frame_piano = cap_piano.read()

    # Stop if either video ends
    if not ret_main or not ret_piano:
        break  

    # Check if frame_piano is valid
    if frame_piano is None:
        print("Error: Source video frame is empty.")
        continue

    # Resize the source video frame
    roi_width = int(np.linalg.norm(ref_pt1 - ref_pt2))
    roi_height = int(np.linalg.norm(ref_pt1 - ref_pt3))
    frame_piano = cv2.resize(frame_piano, (roi_width, roi_height))

    # Define source and destination points
    pts_src = np.array([
        [0, 0], 
        [roi_width, 0], 
        [roi_width, roi_height], 
        [0, roi_height]
    ], dtype=np.float32)

    pts_dst = np.array([
        [ref_pt1[0], ref_pt1[1]],  
        [ref_pt2[0], ref_pt2[1]],  
        [ref_pt3[0], ref_pt3[1]],  
        [ref_pt4[0], ref_pt4[1]]   
    ], dtype=np.float32)

    # Compute homography and warp the frame
    M = cv2.getPerspectiveTransform(pts_src, pts_dst)
    warped_piano = cv2.warpPerspective(frame_piano, M, (frame_main.shape[1], frame_main.shape[0]))

    # Display debug info
    print("Warping frame from piano.mp4")
    
    # Show the result
    cv2.imshow("Warped Video Overlay", warped_piano)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  

cap_main.release()
cap_piano.release()
cv2.destroyAllWindows()


NameError: name 'cv2' is not defined